In [1]:
import netCDF4
import itertools

from tqdm import tqdm

Nx = 512
Ny = 512
Nz = 601
Na = 5
Ns = 512 * 512

In [15]:
mu_id = ['10', '8', '6', '4', '2']

T = np.zeros((Na, Nz, Ny * Nx))
t = np.zeros((Na, Nz, Ny * Nx))

T_ssd = np.zeros((Na, Nz, Ns))
t_ssd = np.zeros((Na, Nz, Ns))

T_300 = np.zeros((Na, Nz, Ns))
t_300 = np.zeros((Na, Nz, Ns))

for i in tqdm(range(Na)):
    
    t[i, :, :] = np.array(netCDF4.Dataset('./ssd/222809/' + mu_id[i] + '/taugrid.222809.nc.1')['tau']).reshape(Nz, Ny * Nx)
    T[i, :, :] = np.array(netCDF4.Dataset('./ssd/222809/' + mu_id[i] + '/T_onTau.222809.nc.1')['T']).reshape(Nz, Ny * Nx)
    
    sample = np.random.choice(Nx * Ny, Ns, replace = False)
    
    t_ssd[i, :, :] = t[i, :, sample].T
    T_ssd[i, :, :] = T[i, :, sample].T

    t[i, :, :] = np.array(netCDF4.Dataset('./300G/627321/' + mu_id[i] + '/taugrid.627321.nc.1')['tau']).reshape(Nz, Ny * Nx)
    T[i, :, :] = np.array(netCDF4.Dataset('./300G/627321/' + mu_id[i] + '/T_onTau.627321.nc.1')['T']).reshape(Nz, Ny * Nx)
    
    sample = np.random.choice(Nx * Ny, Ns, replace = False)
    
    t_300[i, :, :] = t[i, :, sample].T
    T_300[i, :, :] = T[i, :, sample].T

np.savez('intro_img_2', t_ssd = t_ssd, t_300 = t_300, T_ssd = T_ssd, T_300 = T_300)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [03:11<00:00, 38.35s/it]


In [2]:
print('reading t_ssd...')
t_ssd = np.load('intro_img_2.npz')['t_ssd']
print('reading t_300...')
t_300 = np.load('intro_img_2.npz')['t_300']
print('reading T_ssd...')
T_ssd = np.load('intro_img_2.npz')['T_ssd']
print('reading T_ssd...')
T_300 = np.load('intro_img_2.npz')['T_300']

reading t_ssd...
reading t_300...
reading T_ssd...
reading T_ssd...


In [9]:
import itertools

for i, k, j in itertools.product(range(Na), range(Ns), range(Nz - 1)):
            
    delta = np.abs(np.log10(t_ssd[i, j, k]) - np.log10(t_ssd[i, j + 1, k]))
        
    if np.abs(delta - 0.0001) < 1e-6: T_ssd[i, j, k] = np.nan
    
    delta = np.abs(np.log10(t_300[i, j, k]) - np.log10(t_300[i, j + 1, k]))

    if np.abs(delta - 0.0001) < 1e-6: T_300[i, j, k] = np.nan
            
np.savez('intro_img_2_nans', t_ssd = t_ssd, t_300 = t_300, T_ssd = T_ssd, T_300 = T_300)
            
t_ssd = np.load('intro_img_2_nans.npz')['t_ssd']
t_300 = np.load('intro_img_2_nans.npz')['t_300']
T_ssd = np.load('intro_img_2_nans.npz')['T_ssd']
T_300 = np.load('intro_img_2_nans.npz')['T_300']

In [8]:
import itertools

from scipy import interpolate

import sys

def find_interpol_range(tau, Nz, Ns):

    thick = 1e+20
    thinn = 1e-20

    for k in range(Ns):

        opt_thick_end = tau[Nz - 1, k]
        opt_thinn_end = tau[0, k]

        if opt_thinn_end > thinn: thinn = opt_thinn_end
        
        if opt_thick_end < thick: thick = opt_thick_end
    
    return np.log10(thinn), np.log10(thick)

def find_shortest_delta(tau, Nz, Ns):
    
    tau_1d = np.log10(tau.T).reshape(Nz * Ns)
    
    delta = np.abs(np.diff(tau_1d))
    
    idx_min = np.argmin(np.abs(delta - 0.001))
    
#    delta = 20
    
#    for j, k in tqdm(itertools.product(range(Nz - 1), range(Ns)), desc = str(i) + ', ' + cube + ', find shortest delta'):
        
#        d = np.abs(np.log10(tau[j + 1, k]) - np.log10(tau[j, k]))
        
#        if d < delta and d > 0.001: delta = d

#    print(delta[idx_min])

    #print(np.min(np.abs(delta - 0.001)))
    
#    sys.exit()
            
    return delta[idx_min]

def const_interpol_grid(thin, thick, delta):
    
    Ng = int(np.floor(np.abs(thick - thin) / delta))
    
    grid = np.zeros(Ng)
    
    grid[0] = thin
    
    for i in range(Ng - 1): grid[i + 1] = grid[i] + delta
    
    return grid

def interpol_and_average(i, cube, tau, T, igrid, Ns):
    
    T_mean = np.zeros(len(igrid))
    
    for k in tqdm(range(Ns), desc = str(i) + ', ' + cube + ', interpolate and average'):
    
        f = interpolate.interp1d(np.log10(tau[:, k]), T[:, k])
        
        Ti = f(igrid)
        
        T_mean += Ti
        
    T_mean /= Ns
    
    return T_mean

grid_ssd = np.ones((Na, 110000)) * np.nan
grid_300 = np.ones((Na, 110000)) * np.nan

Ti_ssd = np.ones((Na, 110000)) * np.nan
Ti_300 = np.ones((Na, 110000)) * np.nan

for i in range(Na):
    
    thin, thick = find_interpol_range(t_ssd[i, :, :], Nz, Ns)
    delta =       find_shortest_delta(t_ssd[i, :, :], Nz, Ns)
    grid =        const_interpol_grid(thin, thick, delta)
    T_mean =      interpol_and_average(i, 'ssd', t_ssd[i, :, :], T_ssd[i, :, :], grid, Ns)
    
    grid_ssd[i, : len(grid)] = 10.0**grid    
    Ti_ssd[i, : len(T_mean)] = T_mean
    
    thin, thick = find_interpol_range(t_300[i, :, :], Nz, Ns)
    delta =       find_shortest_delta(t_300[i, :, :], Nz, Ns)
    grid =        const_interpol_grid(thin, thick, delta)
    T_mean =      interpol_and_average(i, '300', t_300[i, :, :], T_300[i, :, :], grid, Ns)
    
    grid_300[i, : len(grid)] = 10.0**grid
    Ti_300[i, : len(T_mean)] = T_mean

np.savez('intro_img_interpol', grid_ssd = grid_ssd, grid_300 = grid_300, Ti_ssd = Ti_ssd, Ti_300 = Ti_300)

4, 300, interpolate and average: 100%|█████████████████████████████████████████████████████████████████████████████████████████████| 262144/262144 [01:01<00:00, 4234.10it/s]


In [9]:
print('reading grid_ssd...')
grid_ssd = np.load('intro_img_interpol.npz')['grid_ssd']
print('reading Ti_ssd...')
Ti_ssd =   np.load('intro_img_interpol.npz')['Ti_ssd']
print('reading grid_300...')
grid_300 = np.load('intro_img_interpol.npz')['grid_300']
print('reading Ti_300...')
Ti_300 =   np.load('intro_img_interpol.npz')['Ti_300']

reading grid_ssd...
reading Ti_ssd...
reading grid_300...
reading Ti_300...


In [21]:
import matplotlib.pyplot as plt

import matplotlib.gridspec as gridspec

from tqdm import tqdm

from matplotlib.ticker import LogLocator
from matplotlib.ticker import MultipleLocator
from matplotlib.ticker import AutoMinorLocator

props = dict(boxstyle = 'round', facecolor = 'wheat', alpha = 0.5)

plt.rcParams.update({'font.size': 8})
plt.rcParams["font.family"] = 'Times New Roman'

mus = ['1.0', '0.8', '0.6', '0.4', '0.2']

fig = plt.figure(dpi = 300)

fig.set_size_inches(10, 11.25, forward = True)

fig.tight_layout()

gs = gridspec.GridSpec(5, 3, width_ratios = [1, 1, 1], hspace = 0.2, wspace = 0.2)

for i in tqdm(range(Na)):

    ssd = plt.subplot(gs[i, 0])
    mag = plt.subplot(gs[i, 1])
        
    com = plt.subplot(gs[i, 2])
    
    for k in np.random.choice(Nx * Ny, 32, replace = False):
        
        ssd.plot(t_ssd[i, :, k], T_ssd[i, :, k], color = 'gray')
        mag.plot(t_300[i, :, k], T_300[i, :, k], color = 'gray')
        
    ssd.plot(grid_ssd[i, :], Ti_ssd[i, :], color = 'k')
    mag.plot(grid_300[i, :], Ti_300[i, :], color = 'r')
    
    com.plot(grid_ssd[i, :], Ti_ssd[i, :], color = 'k', label = 'SSD')
    com.plot(grid_300[i, :], Ti_300[i, :], color = 'r', label = '300G')
        
#        if i == 0:
            
#            ssd.axvline(x = thick, color = 'r', linestyle = '--')
#            ssd.axvline(x = thinn, color = 'r', linestyle = '--')
        
    ssd.set_xscale('log')
    
    ssd.set_xlim(3e+1, 2e-12)
    ssd.set_ylim(2000, 10500)
    
    ssd.set_ylabel('Temperature, K')
    
    if i == 0: ssd.set_title('SSD')
    
    mag.set_xscale('log')
    
    mag.set_xlim(ssd.get_xlim())
    mag.set_ylim(ssd.get_ylim())
    
    if i == 0: mag.set_title('300G')
        
    com.set_xscale('log')
    
    com.set_xlim(3e+1, 2e-12)
    com.set_ylim(2000, 10500)
    
    if i == 4: ssd.set_xlabel(r'$\tau_\mathrm{Ross}$')
    if i == 4: mag.set_xlabel(r'$\tau_\mathrm{Ross}$')
    if i == 4: com.set_xlabel(r'$\tau_\mathrm{Ross}$')
    
    com.set_ylabel(r'$\mu =$' + mus[i])
    com.yaxis.set_label_position("right")
    
    if i == 0: com.legend(framealpha = 1, loc = 1, handletextpad = 1, prop = {'size': 7.5})

plt.savefig('intro_img_2.pdf', bbox_inches = 'tight')

plt.close('all')

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00,  6.00it/s]
